In [2]:
# STEP 1 — Explore /schedule endpoint with required parameters

import requests
from datetime import date

url = "https://statsapi.mlb.com/api/v1/schedule"

# Use today's date
today = date.today().isoformat()

params = {
    "sportId": 1,           # 1 = MLB
    "date": today           # or use startDate / endDate if you want a range
}

response = requests.get(url, params=params)

print("HTTP status:", response.status_code)
print("Requested URL:", response.url)


HTTP status: 200
Requested URL: https://statsapi.mlb.com/api/v1/schedule?sportId=1&date=2025-09-05


In [3]:
# STEP 2 — Convert the response to JSON and peek

data = response.json()   # convert raw response into a Python dict

print("Top-level type:", type(data))
print("Top-level keys:", list(data.keys()))


Top-level type: <class 'dict'>
Top-level keys: ['copyright', 'totalItems', 'totalEvents', 'totalGames', 'totalGamesInProgress', 'dates']


In [9]:
# STEP 2 — Inspect the schedule structure

sched = response.json()

# The root keys
print("Top-level keys:", list(sched.keys()))

# Schedule data lives under 'dates'
dates = sched.get("dates", [])
print("Number of date buckets:", len(dates))

if dates:
    first_bucket = dates[0]
    print("Date bucket keys:", list(first_bucket.keys()))

    games = first_bucket.get("games", [])
    print("Number of games on this date:", len(games))

    # Look at keys of the first game
    if games:
        first_game = games[0]
        print("First game keys:", list(first_game.keys()))


Top-level keys: ['copyright', 'totalItems', 'totalEvents', 'totalGames', 'totalGamesInProgress', 'dates']
Number of date buckets: 1
Date bucket keys: ['date', 'totalItems', 'totalEvents', 'totalGames', 'totalGamesInProgress', 'games', 'events']
Number of games on this date: 15
First game keys: ['gamePk', 'gameGuid', 'link', 'gameType', 'season', 'gameDate', 'officialDate', 'status', 'teams', 'venue', 'content', 'gameNumber', 'publicFacing', 'doubleHeader', 'gamedayType', 'tiebreaker', 'calendarEventID', 'seasonDisplay', 'dayNight', 'scheduledInnings', 'reverseHomeAwayStatus', 'inningBreakLength', 'gamesInSeries', 'seriesGameNumber', 'seriesDescription', 'recordSource', 'ifNecessary', 'ifNecessaryDescription']


In [10]:
# STEP — Build a clean list of game records from /schedule

dates = sched.get("dates", [])
clean_games = []

for bucket in dates:
    for g in bucket.get("games", []):
        status = g.get("status", {}) or {}
        teams  = g.get("teams", {}) or {}
        home   = (teams.get("home") or {}).get("team") or {}
        away   = (teams.get("away") or {}).get("team") or {}
        venue  = g.get("venue", {}) or {}

        clean_games.append({
            "gamePk": g.get("gamePk"),
            "officialDate": g.get("officialDate"),          # YYYY-MM-DD (local to game site)
            "gameDate_utc": g.get("gameDate"),              # ISO UTC timestamp
            "status_code": status.get("statusCode"),
            "status_detailed": status.get("detailedState"),
            "gameType": g.get("gameType"),                  # R, S, P, etc.
            "seriesGameNumber": g.get("seriesGameNumber"),
            "seriesDescription": g.get("seriesDescription"),
            "doubleHeader": g.get("doubleHeader"),
            "dayNight": g.get("dayNight"),
            "scheduledInnings": g.get("scheduledInnings"),

            "home_team_id": home.get("id"),
            "home_team_name": home.get("name"),
            "away_team_id": away.get("id"),
            "away_team_name": away.get("name"),

            "venue_id": venue.get("id"),
            "venue_name": venue.get("name"),
        })

print(f"Built {len(clean_games)} game records")
for row in clean_games[:3]:
    print(row)


Built 15 game records
{'gamePk': 776455, 'officialDate': '2025-09-05', 'gameDate_utc': '2025-09-05T18:20:00Z', 'status_code': 'S', 'status_detailed': 'Scheduled', 'gameType': 'R', 'seriesGameNumber': 1, 'seriesDescription': 'Regular Season', 'doubleHeader': 'N', 'dayNight': 'day', 'scheduledInnings': 9, 'home_team_id': 112, 'home_team_name': 'Chicago Cubs', 'away_team_id': 120, 'away_team_name': 'Washington Nationals', 'venue_id': 17, 'venue_name': 'Wrigley Field'}
{'gamePk': 776450, 'officialDate': '2025-09-05', 'gameDate_utc': '2025-09-05T22:40:00Z', 'status_code': 'S', 'status_detailed': 'Scheduled', 'gameType': 'R', 'seriesGameNumber': 1, 'seriesDescription': 'Regular Season', 'doubleHeader': 'N', 'dayNight': 'night', 'scheduledInnings': 9, 'home_team_id': 113, 'home_team_name': 'Cincinnati Reds', 'away_team_id': 121, 'away_team_name': 'New York Mets', 'venue_id': 2602, 'venue_name': 'Great American Ball Park'}
{'gamePk': 776452, 'officialDate': '2025-09-05', 'gameDate_utc': '2025-

In [11]:
# STEP — Convert to DataFrame (this is your /schedule Silver)

import pandas as pd
df_schedule = pd.DataFrame(clean_games)
print("Shape:", df_schedule.shape)
df_schedule.head(10)


Shape: (15, 17)


,gamePk,officialDate,gameDate_utc,status_code,status_detailed,gameType,seriesGameNumber,seriesDescription,doubleHeader,dayNight,scheduledInnings,home_team_id,home_team_name,away_team_id,away_team_name,venue_id,venue_name
0,776455,2025-09-05,2025-09-05T18:20:00Z,S,Scheduled,R,1,Regular Season,N,day,9,112,Chicago Cubs,120,Washington Nationals,17,Wrigley Field
1,776450,2025-09-05,2025-09-05T22:40:00Z,S,Scheduled,R,1,Regular Season,N,night,9,113,Cincinnati Reds,121,New York Mets,2602,Great American Ball Park
2,776452,2025-09-05,2025-09-05T22:40:00Z,S,Scheduled,R,1,Regular Season,N,night,9,116,Detroit Tigers,145,Chicago White Sox,2394,Comerica Park
3,776453,2025-09-05,2025-09-05T22:40:00Z,S,Scheduled,R,1,Regular Season,N,night,9,134,Pittsburgh Pirates,158,Milwaukee Brewers,31,PNC Park
4,776454,2025-09-05,2025-09-05T23:05:00Z,S,Scheduled,R,1,Regular Season,N,night,9,110,Baltimore Orioles,119,Los Angeles Dodgers,2,Oriole Park at Camden Yards
5,776457,2025-09-05,2025-09-05T23:05:00Z,S,Scheduled,R,1,Regular Season,N,night,9,147,New York Yankees,141,Toronto Blue Jays,3313,Yankee Stadium
6,776456,2025-09-05,2025-09-05T23:10:00Z,S,Scheduled,R,1,Regular Season,N,night,9,146,Miami Marlins,143,Philadelphia Phillies,4169,loanDepot park
7,776460,2025-09-05,2025-09-05T23:15:00Z,S,Scheduled,R,1,Regular Season,N,night,9,144,Atlanta Braves,136,Seattle Mariners,4705,Truist Park
8,776451,2025-09-05,2025-09-05T23:35:00Z,S,Scheduled,R,2,Regular Season,N,night,9,139,Tampa Bay Rays,114,Cleveland Guardians,2523,George M. Steinbrenner Field
9,776446,2025-09-05,2025-09-05T23:40:00Z,S,Scheduled,R,1,Regular Season,N,night,9,118,Kansas City Royals,142,Minnesota Twins,7,Kauffman Stadium
